In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

# Test area #

In [4]:
df=pd.read_excel("facList.xlsx")
df=df.loc[:,['ทะเบียนโรงงาน ','ชื่อโรงงาน ']]
df['newCode']=""
df['facType']=""
df['ISIC']=""
df['TSIC']=""
df['desc']=""
df.head(10)

,ทะเบียนโรงงาน,ชื่อโรงงาน,newCode,facType,ISIC,TSIC,desc
0,ศ3-20(1)-1/34ชม,บริษัท เชียงใหม่ เบเวอเรช จำกัด,,,,,
1,ศ3-8(1)-4/36ชม,บริษัท สหปราจีนบุรีอุตสาหกรรมอาหาร จำกัด,,,,,
2,ศ3-8(1)-2/36ชม,บริษัท ช้างอวอดส์ (1959) จำกัด,,,,,
3,ศ3-8(1)-1/34ชม,บริษัท สันติภาพ (ฮั่วเพ้ง 1958) จำกัด,,,,,
4,3-58(1)-119/50ชม,บริษัท ไดมอนด์ วัสดุ จำกัด,,,,,
5,ศ3-8(2)-2/33ชร,บริษัท แพนสตาร์ จำกัด,,,,,
6,จ3-1-3/19ชร,สถานีบ่มใบยาสูบปงของ,,,,,
7,ศ3-15(1)-2/38พบ,บริษัท โกรเบสท์คอร์โพเรชั่น จำกัด,,,,,
8,3-44-1/40พบ,บริษัท กังวาลโพลีเอสเตอร์ จำกัด,,,,,
9,ศ3-15(1)-3/38พบ,บริษัท ลีพัฒนาผลิตภัณฑ์ จำกัด (มหาชน),,,,,


In [3]:
# Open Chrome
driver = webdriver.Chrome('chromedriver.exe')

# Open URL
driver.get('https://www.diw.go.th/hawk/content.php?mode=data1search')

In [5]:
facName="สถานีบ่มใบยาสูบปงของ"
facCode="จ3-1-3/19ชร"

In [6]:
driver.find_element_by_name('code').clear()
driver.find_element_by_name('code').send_keys(facCode)

In [7]:
driver.find_element_by_name('Name').clear()
driver.find_element_by_name('Name').send_keys(facName)

In [8]:
driver.find_element_by_name('Submit').click()

In [9]:
window_after = driver.window_handles[1]
#switch on to new child window
driver.switch_to.window(window_after)

In [10]:
soup = BeautifulSoup(driver.page_source,'html')
tables = soup.find_all('table')

In [11]:
#print(tables)

In [12]:
table=tables[0]

In [13]:
for tr in soup.find_all('tr')[1:2]:
    tds = tr.find_all('td')
    for i in tds:
        print(i.text.replace('\n','').replace('\r','').replace('\t','').strip())
        print("*-----------------------------------------------*")

สถานีบ่มใบยาสูบปงของ20570000325199 [จ3-1-3/19ชร]ประเภทโรงงาน 1ISIC 1200 TSIC 12001
*-----------------------------------------------*
323 ม.5ถ.- ต.บ้านแซวอ.เชียงแสน จ.เชียงราย   57150
*-----------------------------------------------*
บริษัท เทพวงศ์ จำกัด436ถ.เชียงใหม่-ลำปาง ต.หนองป่าครั่งอ.เมืองเชียงใหม่ จ.เชียงใหม่   50000
*-----------------------------------------------*
บ่มใบยาสูบ จำนวน 50 เตาเครื่องจักร  799.34 HPเงินทุน 87,500,000 บาทคนงาน 30 คน
*-----------------------------------------------*


In [14]:
tr=soup.find_all('tr')[1]
td1=tr.find_all('td')[0]

In [15]:
td1_text=td1.text.replace('\n','').replace('\r','').replace('\t','').strip()

In [16]:
facCodeStart=td1_text.find(facName)+len(facName)

In [17]:
facCode=td1_text[facCodeStart:facCodeStart+14]

In [18]:
print(facCode)

20570000325199


In [19]:
tr=soup.find_all('tr')[1]
td2=tr.find_all('td')[3]
td2_text=td2.text.replace('\n','').replace('\r','').replace('\t','').strip()
print(td2_text)
print(td2_text[0:td2_text.find("เครื่องจักร")])

บ่มใบยาสูบ จำนวน 50 เตาเครื่องจักร  799.34 HPเงินทุน 87,500,000 บาทคนงาน 30 คน
บ่มใบยาสูบ จำนวน 50 เตา


In [21]:
tr=soup.find_all('tr')[1]
td3=tr.find_all('td')[1]
td3_text=td3.text.replace('\n','').replace('\r','').replace('\t','').strip()

print(td3_text[0:td3_text.find("โทร.")])
if "โทร." in td3_text:
    print(td3_text[td3_text.find("โทร.")+4:].strip())

323 ม.5ถ.- ต.บ้านแซวอ.เชียงแสน จ.เชียงราย   5715


In [65]:
window_before = driver.window_handles[0]
#switch on to new child window
driver.switch_to.window(window_before)

# Deploy area #

In [64]:
df=pd.read_excel("facList.xlsx")
df=df.loc[:,['ทะเบียนโรงงาน ','ชื่อโรงงาน ']]
df[['newCode','facType','ISIC','TSIC','desc','add','tel']]="-"
#df=df.loc[0:6,:]
#df

In [61]:
# Open Chrome
driver = webdriver.Chrome('chromedriver.exe')

# Open URL
driver.get('https://www.diw.go.th/hawk/content.php?mode=data1search')

In [39]:
def searchFac(facCode,facName,driver):
    switchWin(0,driver)
    driver.find_element_by_name('code').clear()
    driver.find_element_by_name('code').send_keys(facCode)
    driver.find_element_by_name('Name').clear()
    driver.find_element_by_name('Name').send_keys(facName)
    driver.find_element_by_name('Submit').click()
    return

def switchWin(page,driver):
    window_after = driver.window_handles[page]
    driver.switch_to.window(window_after)
    return


In [40]:
def scrapData(tr,facCode,facName):
    td1=tr.find_all('td')[0]
    td1_text=td1.text.replace('\n','').replace('\r','').replace('\t','').strip()
    td2=tr.find_all('td')[3]
    td2_text=td2.text.replace('\n','').replace('\r','').replace('\t','').strip()
    td3=tr.find_all('td')[1]
    td3_text=td3.text.replace('\n','').replace('\r','').replace('\t','').strip()

    facCodeStart=td1_text.find(facName)+len(facName)
    facTypeStart=td1_text.find("ประเภทโรงงาน")+len("ประเภทโรงงาน")+1
    facIsicStart=td1_text.find("ISIC")+len("ISIC")+1
    facTsicStart=td1_text.find("TSIC")+len("ISIC")+1

    facCode=td1_text[facCodeStart:facCodeStart+14]
    facType=td1_text[facTypeStart:td1_text.find("ISIC")].strip()
    facIsic=td1_text[facIsicStart:facIsicStart+4]
    facTsic=td1_text[facTsicStart:facTsicStart+5]
    facDesc=td2_text[0:td2_text.find("เครื่องจักร")]
    facAdd=td3_text
    facTel=""

    if "โทร." in td3_text:
        facTel=td3_text[td3_text.find("โทร.")+4:].strip()
        facAdd=td3_text[0:td3_text.find("โทร.")]
        
    return facCode,facType,facIsic,facTsic,facDesc,facAdd,facTel

## Start loop

In [65]:
count_err=0
for i in df.index:
    
    facName=df.iloc[i,1].strip()
    facCode=df.iloc[i,0].strip()
    searchFac(facCode,"",driver)
    switchWin(1,driver)
    
    try :
        soup = BeautifulSoup(driver.page_source,'html')
        tr=soup.find_all('tr')[1]
        df.loc[i,'newCode'],df.loc[i,'facType'],df.loc[i,'ISIC'],df.loc[i,'TSIC'],df.loc[i,'desc'],df.loc[i,'add'],df.loc[i,'tel']=scrapData(tr,facCode,facName)
    except :
        #print(f"ทะเบียนโรงงานเก่า {df.iloc[i,0]} ชื่อโรงงาน {df.iloc[i,1]}")
        count_err+=1
    switchWin(0,driver)
print()

ทะเบียนโรงงานเก่า ศ3-8(2)-2/33ชร  ชื่อโรงงาน บริษัท แพนสตาร์ จำกัด 
ทะเบียนโรงงานเก่า 3-15(1)-4/51นพ  ชื่อโรงงาน บริษัท โกรเบสท์คอร์โพเรชั่น จำกัด 
ทะเบียนโรงงานเก่า 3-9(6)-1/32อย  ชื่อโรงงาน บริษัท นครหลวง อินฟินนิทิเกรน จำกัด 
ทะเบียนโรงงานเก่า ศ3-10(3)-1/34รบ  ชื่อโรงงาน บริษัท นาเชา (ประเทศไทย) จำกัด 
ทะเบียนโรงงานเก่า ศ3-8(1)-1/35รบ  ชื่อโรงงาน บริษัท ฮาร์โมนี่ อะกริ บิสสิเนส จำกัด 
ทะเบียนโรงงานเก่า 3-22(2)-30/15สป  ชื่อโรงงาน บริษัท ศิลปเสนีพาณิชย์ จำกัด 
ทะเบียนโรงงานเก่า 3-22(2)-11/21สป  ชื่อโรงงาน บริษัท ศิลปเสนีพาณิชย์ จำกัด 
ทะเบียนโรงงานเก่า 3-22(3)-3/34สป  ชื่อโรงงาน บริษัท ไทยฟิลาเมนท์ฟินิชชิ่ง จำกัด 
ทะเบียนโรงงานเก่า 3-22(3)-3/28สป  ชื่อโรงงาน บริษัท โกลเดิ้นไทย อินดัสทรียส์ จำกัด 


In [66]:
print(count_err)

9


In [67]:
df.to_csv("scrapData.csv")